# 2등 JM Unet

## #import libraries

In [ ]:
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import glob
import gc
import random
from tqdm import tqdm
import time
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split,KFold
from sklearn.metrics import f1_score, accuracy_score, mean_absolute_error, confusion_matrix, recall_score, precision_score
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint  
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input,BatchNormalization,LayerNormalization,Dropout,Conv2D,Conv2DTranspose, MaxPooling2D, Flatten,Activation, PReLU,concatenate, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Activation
from tensorflow.keras.utils import multi_gpu_model

os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2" # 사용할 GPU 설정

## # evaluation metric

In [ ]:
# Metrics
# f1 score 계산
def fscore(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    y_true = y_true.reshape(1, -1)[0]
    y_pred = y_pred.reshape(1, -1)[0]
    remove_NAs = y_true >= 0
    y_true = np.where(y_true[remove_NAs] >= 0.1, 1, 0)
    y_pred = np.where(y_pred[remove_NAs] >= 0.1, 1, 0)
    return(f1_score(y_true, y_pred))
def fscore_keras(y_true, y_pred):
    score = tf.py_function(func=fscore, inp=[y_true, y_pred], Tout=tf.float32, name='fscore_keras')
    return score

# MAE/f1
def maeOverFscore(y_true, y_pred):
    return AIF_mae(y_true, y_pred) / (fscore(y_true, y_pred) + 1e-07)
def maeOverFscore_keras(y_true, y_pred):
    score = tf.py_function(func=maeOverFscore, inp=[y_true, y_pred], Tout=tf.float32,  name='custom_mse') 
    return score

# 실제값 0.1이상인 것에대한 MAE 값 계산
def AIF_mae(y_true, y_pred) :
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    y_true = y_true.reshape(1, -1)[0]
    y_pred = y_pred.reshape(1, -1)[0]
    over_threshold = y_true >= 0.1
    return np.mean(np.abs(y_true[over_threshold] - y_pred[over_threshold]))

def mae_custom(y_true, y_pred):
    score = tf.py_function(func=AIF_mae, inp=[y_true, y_pred], Tout=tf.float32,  name='mae_custom') 
    return score

# classification 모델의 fscore 계산
def fscore_(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    y_true = y_true.reshape(1, -1)[0]
    y_pred = y_pred.reshape(1, -1)[0]
    y_true = np.where(y_true >= 0.5, 1, 0)
    y_pred = np.where(y_pred >= 0.5, 1, 0)
    return(f1_score(y_true, y_pred))

def fscore_classification(y_true, y_pred):
    score = tf.py_function(func=fscore_, inp=[y_true, y_pred], Tout=tf.float32, name='fscore_classification')
    return score

## # data load & preprocessing

In [ ]:
def data_load(files, TRAIN=True):
    '''
    TRAIN : {True:훈련 데이터 로드, 결측값이 없는 온전한 데이터만 로드, False:테스트 데이터 로드, 모든 데이터 로드}
    '''
    tmp=[]
    for file in tqdm(files[:]):
        data = np.load(file).astype(np.float32)
        if TRAIN:
            if (data[:,:,-1]<0).sum()==0:                     # 결측값이 없는 데이터만 로드
                tmp.append(data)
        else:
            tmp.append(data)
    return np.array(tmp)

In [ ]:
# 결측값이 없는 데이터만 Load
train_files = sorted(glob.glob('data/train/*.npy'))[:]
train_ = data_load(train_files)
print('train shape :', train_.shape)

## # Exploratory data analysis

In [ ]:
'''
한 픽셀의 target 값이 0.1 이상인 행들만 선택하여 밝기온도 차와 target의 상관계수가 높은 순으로 시각화 하여 확인
'''

In [ ]:
# 데이터의 상관관계를 확인하기 위해서 먼저 pandas dataframe 형태로 만들어 준다.
# dataframe으로 만들기 위해 reshape하는 부분이 endgame님 코드와 조금 다르다. 어쨌든 큰 램 메모리 용량이 필요할듯하다.
df = pd.DataFrame(train_.reshape(-1, 15))
df.columns=['BT1', 'BT2', 'BT3', 'BT4', 'BT5', 'BT6', 'BT7', 'BT8', 'BT9','type','GMI_lon', 'GMI_lat', 'DPR_lon', 'DPR_lat','target']
BT_column=['BT1', 'BT2', 'BT3', 'BT4', 'BT5', 'BT6', 'BT7', 'BT8', 'BT9']

# 밝기온도 차 컬럼 추가
for i in range(9):
    for j in range(i+1, 9):
        df['{}_{}_diff'.format(BT_column[i],BT_column[j])] = df[BT_column[i]] - df[BT_column[j]]

# target 값이 0.1 이상인 값만 추출
df = df[df['target']>=0.1]

# 상관계수 계산
df_corr = df.corr()

# 강수량 (target data)과 가장 큰 상관관계를 보이는 feature를 찾으면 된다.
# 상관관계 값에 따라 가중치를 두는 것도 고려해볼만 할듯하다
col = df_corr['target'].abs().sort_values(ascending=False).index

# 시각화
# 먼저 figure size를 정해줘야 한다.
plt.rcParams['figure.figsize'] = [15, 1]
for idx in range(0, 36, 9):
    # value range가 엄청 넓기 때문에 범위를 지정해 줄 필요가 있을듯 하다. vmin=0, vmax=0.5
    # sns.heatmap(데이터 전체, 우리가 주로 보고자 하는 행)
    sns.heatmap(df_corr.loc[['target'],[k for k in col if 'diff' in k][idx:idx+9]].abs(), annot=True, linewidths=1, vmin=0, vmax=0.5)
    
    #  plt.xticks = 지정할 눈금들
    plt.xticks(rotation=20)
    plt.show()

In [ ]:
'''
GMI(경도,위도) plot에서의 강수량을 시각화 하여 확인. (강수량 수치가 클수록 색이 진하고 크기가 큰 점)
강수량이 중구난방 있지않고 비교적 밀집되어 있음을 알 수 있다. --> CNN 기반 딥러닝 모델 학습이 용이할 것 같다. --> Unet 모델 사용

뭔가 뭉쳐있는 데이터를 K mean clustering으로 더 잘 featuring 할 수 있지 않을까 하는 생각도 든다.
'''

In [ ]:
def visualize(idx):
    plt.figure(figsize=(10,10))
    pal = sns.dark_palette("palegreen", as_cmap=True)
    colormap = plt.cm.RdBu
    sns.scatterplot(train_[idx,:,:,10].reshape(-1), train_[idx,:,:,11].reshape(-1), hue=train_[idx,:,:,-1].reshape(-1), size=train_[idx,:,:,-1].reshape(-1))
    plt.xlabel('latitude')
    plt.ylabel('longitude')
    plt.show()

In [ ]:
visualize(9502)
visualize(9503)
visualize(9504)
visualize(9505)

## # modeling

In [ ]:
'''
Feature Engineering & Initial Modeling
변수 선택 및 feature 생성 함수
밝기온도 차이와 target 사이에서 상관계수가 높은 24개를 선택

[(3,8), (5,8), (1,4), (6, 8), (5, 9), (3, 9), (4, 9), (4, 8), (7, 9), (7,8), (6, 9), (4, 6), (3, 6), (4, 5), (3, 4),(6, 7), (1, 8), (3, 5), (4, 7), (1, 9), (2, 8), (5, 7), (2, 9), (2, 4)]

지표타입 feature은 get_dummy를 통해 one-hot encoding (이 부분도 특이한 듯. categorical feature도 활용)
'''

In [ ]:
def gen_feature(data, type_encoding=True):
    #  밝기온도와 target의 상관 계수가 높은 24개를 생성.
    '''
    type_encoding : 지표 타입을 get_dummy를 사용하여 인코딩 할지 여부.
    '''
    tmp = []
    # 위에 사용한 데이터를 쓰지 않고 여기서 새로 데이터 feature를 만들었다.
    for bt in tqdm([(3,8), (5,8), (1,4), (6, 8), (5, 9), (3, 9), (4, 9), (4, 8), (7, 9), (7,8), (6, 9), (4, 6), (3, 6), (4, 5), (3, 4),
              (6, 7), (1, 8), (3, 5), (4, 7), (1, 9), (2, 8), (5, 7), (2, 9), (2, 4)]):
        tmp.append(data[:,:,:,bt[0]-1]-data[:,:,:,bt[1]-1])
    tmp = np.array(tmp).reshape(-1, 40, 40, 24)

    # type one-hot-encoding
    if type_encoding:
        TYPE = pd.get_dummies((data[:,:,:,9]//100).reshape(-1)).values
        TYPE = TYPE.reshape(-1,40,40,4)
        data = np.append(data[:,:,:,:9], data[:,:,:,10:],axis=-1)            # 원래 type을 제외
        data = np.append(TYPE, data, axis=-1)                                 # 타입 합치기
        data = np.append(tmp,data, axis=-1)                                   # 밝기온도 차이 합치기
    else:
        data = np.append(tmp,data, axis=-1)                                   
    return data

In [ ]:
train_type_enc = gen_feature(train_, type_encoding=True)
train_non_type_enc = gen_feature(train_, type_encoding=False)

In [ ]:
train_type_enc.shape, train_non_type_enc.shape

In [ ]:
del train_
gc.collect()

In [ ]:
'''
Data Agumentation 전 train과 valid 데이터 index로 사전 분리 (5Fold)
train_idx, valid_idx로 저장해두고 훈련 시 불러와 사용.
'''

In [ ]:
# Kfold split
# 다양한 시드를 사용한 모델 결과값을 이용해서 정규화 효과를 노린다.
skf_seed42 = KFold(n_splits=5, shuffle= True, random_state=42)                 # kfold, random_state 설정
skf_seed1030 = KFold(n_splits=5, shuffle= True, random_state=1030)                 # kfold, random_state 설정
skf_seed1234 = KFold(n_splits=5, shuffle= True, random_state=1234)                 # kfold, random_state 설정

kfolds_seed42 = []
kfolds_seed1030 = []
kfolds_seed1234 = []

for train_idx, test_idx in skf_seed42.split(range(len(train_type_enc))):
    kfolds_seed42.append((train_idx, test_idx))
for train_idx, test_idx in skf_seed1030.split(range(len(train_type_enc))):
    kfolds_seed1030.append((train_idx, test_idx))
for train_idx, test_idx in skf_seed1234.split(range(len(train_type_enc))):
    kfolds_seed1234.append((train_idx, test_idx))

In [ ]:
'''
Data Augmentation
target에서 0.1 이상인 픽셀이 50개 이상일 경우 Data Augmentation 한다.
horizontal flip
vertical flip
rotation 90, 180 270
vertical flip + rotation90
vertical flip + rotation270
증량된 데이터는 기존 train 데이터와 합치고 섞는다.
'''

In [ ]:
def augmentation(X, y):
    tmp = []
    for idx in range(len(X)):
        if (y[idx]>=0.1).sum()>=50:
            tmp.append(idx)

    X_train_aug90 = np.rot90(X[tmp], k=1, axes=(1,2))
    y_train_aug90 = np.rot90(y[tmp], k=1, axes=(1,2))
    X_train_aug180 = np.rot90(X[tmp], k=2, axes=(1,2))
    y_train_aug180 = np.rot90(y[tmp], k=2, axes=(1,2))
    X_train_aug270 = np.rot90(X[tmp], k=3, axes=(1,2))
    y_train_aug270 = np.rot90(y[tmp], k=3, axes=(1,2))
    X_train_flipV = np.flip(X[tmp],1)
    y_train_flipV = np.flip(y[tmp],1)
    X_train_flipH = np.flip(X[tmp],2)
    y_train_flipH = np.flip(y[tmp],2)
    X_train_flipV_90 = np.rot90(X_train_flipV, k=1, axes=(1,2))
    y_train_flipV_90 = np.rot90(y_train_flipV, k=1, axes=(1,2))
    X_train_flipV_270 = np.rot90(X_train_flipV, k=3, axes=(1,2))
    y_train_flipV_270 = np.rot90(y_train_flipV, k=3, axes=(1,2))

    # 증량 시킨 데이터들을 concatenate
    X = np.concatenate([X, X_train_aug90, X_train_aug180, X_train_aug270, X_train_flipV, X_train_flipH, X_train_flipV_90, X_train_flipV_270])
    y = np.concatenate([y, y_train_aug90, y_train_aug180, y_train_aug270, y_train_flipV, y_train_flipH, y_train_flipV_90, y_train_flipV_270])
    del X_train_aug90, y_train_aug90, X_train_aug180, y_train_aug180, X_train_aug270, y_train_aug270, X_train_flipV, y_train_flipV, X_train_flipH, y_train_flipH, X_train_flipV_90, y_train_flipV_90, X_train_flipV_270, y_train_flipV_270
    del tmp
    # agumentation data와 기존 train 데이터 합친것을 shuffle 하여 데이터들을 섞어준다.
    idx = np.array([k for k in range(len(X))]) 
    np.random.seed(1234)    
    np.random.shuffle(idx)
    X = X[idx]
    y = y[idx]
    return X, y

In [ ]:
'''
Model Tuning & Evaluation
기본 Unet 구조를 사용하고 앞과 뒤 층을 수정
input(40x40x41)을 UpSampling 하여 (80x80x41)을 Unet 기본 구조에 입력으로 한다.
마지막 층(80x80x1)에서 Maxpooling하여 DownSampling 하고 (40x40x1)을 output으로 한다.
Epochs : 100
Batch Size : 456
initial Learning rate : 0.001
Learning rate Scheduler : Polynomial Decay (Cosine decay restarts도 번갈아 가며 사용하여 ensemble)
Optimizer : Adam (SGD는 수렴속도가 느려 사용에 어려움이 있었음)
Activation function : ELU 혹은 ReLU를 사용
Output : Regression or Classification
Upsampling Layer : Nearest Neighbor interporation
dropout : 0.5
'''

In [ ]:
strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1", "/gpu:2"])  # multi GPU 설정

In [ ]:
def unet_regression(input_shape,activation='elu', weight=False):
    '''
    activation : relu or elu
    weight : 로드할 모델 가중치의 여부
    '''
    drop = 0.5
    with strategy.scope():
        unit1 = 64
        unit2 = 128
        unit3 = 256
        unit4 = 512
        unit5 = 1024

        inputs = Input(shape=(input_shape))
        inputs_ = UpSampling2D(size=(2,2))(inputs)
        conv1 = Conv2D(unit1, 3, padding = 'same', kernel_initializer = 'he_normal')(inputs_)
        conv1 = BatchNormalization()(conv1)
        conv1 = Activation(activation)(conv1)
        conv1 = Conv2D(unit1, 3, padding = 'same', kernel_initializer = 'he_normal')(conv1)
        conv1 = BatchNormalization()(conv1)
        conv1 = Activation(activation)(conv1)
        pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
        pool1 = Dropout(drop)(pool1)
        conv2 = Conv2D(unit2, 3, padding = 'same', kernel_initializer = 'he_normal')(pool1)
        conv2 = BatchNormalization()(conv2)
        conv2 = Activation(activation)(conv2)
        conv2 = Conv2D(unit2, 3, padding = 'same', kernel_initializer = 'he_normal')(conv2)
        conv2 = BatchNormalization()(conv2)
        conv2 = Activation(activation)(conv2)
        pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
        pool2 = Dropout(drop)(pool2)
        conv3 = Conv2D(unit3, 3, padding = 'same', kernel_initializer = 'he_normal')(pool2)
        conv3 = BatchNormalization()(conv3)
        conv3 = Activation(activation)(conv3)
        conv3 = Conv2D(unit3, 3, padding = 'same', kernel_initializer = 'he_normal')(conv3)
        conv3 = BatchNormalization()(conv3)
        conv3 = Activation(activation)(conv3)
        pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
        pool3 = Dropout(drop)(pool3)
        conv4 = Conv2D(unit4, 3, padding = 'same', kernel_initializer = 'he_normal')(pool3)
        conv4 = BatchNormalization()(conv4)
        conv4 = Activation(activation)(conv4)
        conv4 = Conv2D(unit4, 3, padding = 'same', kernel_initializer = 'he_normal')(conv4)
        conv4 = BatchNormalization()(conv4)
        conv4 = Activation(activation)(conv4)
        drop4 = Dropout(drop)(conv4)
        pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)
        
        conv5 = Conv2D(unit5, 3, padding = 'same', kernel_initializer = 'he_normal')(pool4)
        conv5 = BatchNormalization()(conv5)
        conv5 = Activation(activation)(conv5)
        conv5 = Conv2D(unit5, 3, padding = 'same', kernel_initializer = 'he_normal')(conv5)
        conv5 = BatchNormalization()(conv5)
        conv5 = Activation(activation)(conv5)
        drop5 = Dropout(drop)(conv5)

        up6 = Conv2D(unit4, 2, padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
        up6 = BatchNormalization()(up6)
        up6 = Activation(activation)(up6)
        merge6 = concatenate([drop4,up6], axis = 3)
        merge6 = Dropout(drop)(merge6)
        conv6 = Conv2D(unit4, 3, padding = 'same', kernel_initializer = 'he_normal')(merge6)
        conv6 = BatchNormalization()(conv6)
        conv6 = Activation(activation)(conv6)
        conv6 = Conv2D(unit4, 3, padding = 'same', kernel_initializer = 'he_normal')(conv6)
        conv6 = BatchNormalization()(conv6)
        conv6 = Activation(activation)(conv6)
        up7 = Conv2D(unit3, 2, padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
        up7 = BatchNormalization()(up7)
        up7 = Activation(activation)(up7)
        merge7 = concatenate([conv3,up7], axis = 3)
        merge7 = Dropout(drop)(merge7)
        conv7 = Conv2D(unit3, 3, padding = 'same', kernel_initializer = 'he_normal')(merge7)
        conv7 = BatchNormalization()(conv7)
        conv7 = Activation(activation)(conv7)
        conv7 = Conv2D(unit3, 3, padding = 'same', kernel_initializer = 'he_normal')(conv7)
        conv7 = BatchNormalization()(conv7)
        conv7 = Activation(activation)(conv7)
        up8 = Conv2D(unit2, 2, padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
        up8 = BatchNormalization()(up8)
        up8 = Activation(activation)(up8)
        merge8 = concatenate([conv2,up8], axis = 3)
        merge8 = Dropout(drop)(merge8)
        conv8 = Conv2D(unit2, 3, padding = 'same', kernel_initializer = 'he_normal')(merge8)
        conv8 = BatchNormalization()(conv8)
        conv8 = Activation(activation)(conv8)
        conv8 = Conv2D(unit2, 3, padding = 'same', kernel_initializer = 'he_normal')(conv8)
        conv8 = BatchNormalization()(conv8)
        conv8 = Activation(activation)(conv8)

        up9 = Conv2D(unit1, 2, padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
        up9 = BatchNormalization()(up9)
        up9 = Activation(activation)(up9)
        merge9 = concatenate([conv1,up9], axis = 3)
        merge9 = Dropout(drop)(merge9)
        conv9 = Conv2D(unit1, 3, padding = 'same', kernel_initializer = 'he_normal')(merge9)
        conv9 = BatchNormalization()(conv9)
        conv9 = Activation(activation)(conv9)
        conv9 = Conv2D(unit1, 3, padding = 'same', kernel_initializer = 'he_normal')(conv9)
        conv9 = BatchNormalization()(conv9)
        conv9 = Activation(activation)(conv9)
        conv9 = Conv2D(2, 3, padding = 'same', kernel_initializer = 'he_normal')(conv9)
        conv9 = BatchNormalization()(conv9) 
        conv9 = Activation(activation)(conv9)
        conv9 = MaxPooling2D(pool_size=(2, 2))(conv9)
        conv9 = BatchNormalization()(conv9)
        # regression output
        regression = Conv2D(1, 1, activation = 'relu', name='regression')(conv9)
        model = Model(inputs, regression)
        if weight==False:
            model.compile(loss='mae', optimizer=opt, metrics=[fscore_keras, maeOverFscore_keras, mae_custom] )
    return model